In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import (accuracy_score, f1_score, precision_score, 
                             recall_score, roc_auc_score, mean_squared_error, r2_score)
from sklearn.model_selection import train_test_split
import xgboost as xgb
import math
import shap
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt




In [ ]:

path1 ='/Users/jerzybala/Desktop/BrainBaseMHQ/MHQ_raw_Jan2023_to_Feb2024_processed.csv'

df = pd.read_csv(path1, low_memory=False)